# 🚀 Prática com Transformers + Pydantic + Gráficos

Este notebook inclui:

- Instalação das bibliotecas necessárias
- Carregamento de modelo open-source
- Extração de dados de texto
- Validação de dados com Pydantic
- Visualização em gráfico

Ideal para estudar LLMs sem pagar por APIs ou precisar de PC potente.


In [ ]:
!pip install transformers accelerate sentencepiece pydantic matplotlib

In [ ]:
from transformers import pipeline
from pydantic import BaseModel
import matplotlib.pyplot as plt
import json


In [ ]:
# Modelo para validar dados extraídos
class Macas(BaseModel):
    vermelhas: int
    verdes: int


In [ ]:
pipe = pipeline(
    "text-generation",
    model="microsoft/Phi-3-mini-4k-instruct"
)


In [ ]:
import re
import json
from pydantic import BaseModel

# Modelo Pydantic
class Macas(BaseModel):
    vermelhas: int
    verdes: int

# Texto de entrada
texto = "Haviam 5 maçãs vermelhas e 4 verdes."

# Criar prompt pedindo JSON puro
prompt = f"""
Extraia os dados no formato JSON SOMENTE com campos 'vermelhas' e 'verdes' sem texto adicional:
{texto}
"""

# Gerar resposta do modelo
resposta = pipe(prompt, max_new_tokens=100, return_full_text=False)
saida_texto = resposta[0]["generated_text"]

print("Texto bruto gerado pelo modelo:")
print(saida_texto)

# ✅ Extrair apenas o JSON do texto usando regex
json_match = re.search(r"\{.*\}", saida_texto, re.DOTALL)

if json_match:
    json_texto = json_match.group(0)
    try:
        dados_dict = json.loads(json_texto)
        print("\nDados convertidos para dict:", dados_dict)

        # Validar com Pydantic
        macas = Macas(**dados_dict)
        print("\nDados validados com Pydantic:", macas)

    except json.JSONDecodeError as e:
        print("Erro ao converter em JSON:", e)
else:
    print("Não foi encontrado JSON válido na saída do modelo.")


In [ ]:
try:
    macas = Macas(**dados_dict)
    print("Dados validados com sucesso:", macas)
except Exception as e:
    print("Erro na validação:", e)


In [ ]:
# Preparar dados para o gráfico
dados_grafico = {"Vermelhas": macas.vermelhas, "Verdes": macas.verdes}

plt.bar(dados_grafico.keys(), dados_grafico.values(), color=['red', 'green'])
plt.title("Quantidade de Maçãs por Cor")
plt.ylabel("Quantidade")
plt.show()


## Como funciona

1. O modelo open-source (Phi-3-mini) é carregado no Colab.
2. Pedimos para o modelo **extrair informações do texto** no formato JSON.
3. Convertendo o texto JSON em dicionário Python.
4. Validamos os dados com **Pydantic**.
5. Por fim, plotamos o gráfico usando **Matplotlib**.

Isso permite transformar **texto livre em dados estruturados e visualizações**, tudo de graça e sem sobrecarregar o PC.

